# Tutorial 2: The Information Ordering

*Partial Orders and the Bottom Element*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/02-the-information-ordering.ipynb)

---

## A Note from the Archives

> *Year 857, Capital Archives*
>
> Arren Mott stood before the Classification Integrity Committee, presenting an idea that would reshape the passage tradition.
>
> "Consider," he said, "two computations. The first returns 5. The second loops forever. We have been taught to see the second as failed — as broken, as dangerous."
>
> He paused. "But I propose we see it differently. The second computation simply has *less information* than the first. It tells us nothing about which number it would return. This is not failure — this is *absence*."
>
> "And between these extremes," Mott continued, "lies a rich structure. A partial list [1,2,⊥] has more information than [1,⊥] but less than [1,2,3]. Values can be *ordered* by how much they tell us."
>
> This was EV-857-001: the proposal of the information ordering.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define partial orders and the approximation relation ⊑
2. Explain the meaning of ⊥ (bottom)
3. Distinguish flat domains from non-flat domains
4. Construct lazy list domains with partial information
5. Identify maximal elements in a domain

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Domain elements
domains_df = pd.read_csv(BASE_URL + "domain_elements.csv")

print(f"Loaded {len(domains_df)} domain elements")

## 1. What is a Partial Order?

A **partial order** is a binary relation ⊑ that is:

1. **Reflexive**: x ⊑ x (everything approximates itself)
2. **Antisymmetric**: if x ⊑ y and y ⊑ x, then x = y
3. **Transitive**: if x ⊑ y and y ⊑ z, then x ⊑ z

We read x ⊑ y as "x approximates y" or "x has less information than y".

In [ ]:
# Explore the domain elements
print("Domain elements ordered by information level:")
print(domains_df[['domain', 'element', 'information_level', 'is_bottom', 'is_maximal']].to_string(index=False))

## 2. The Bottom Element: ⊥

Every domain has a **bottom element** ⊥ (pronounced "bottom") that represents:
- No information
- Undefined value
- Non-terminating computation

For any element x: **⊥ ⊑ x**

Bottom is the least element — it approximates everything.

In [ ]:
# Find all bottom elements
bottoms = domains_df[domains_df['is_bottom'] == True]

print("Bottom elements in each domain:")
for _, row in bottoms.iterrows():
    print(f"  {row['domain']}: {row['element']} — {row['notes']}")

## 3. Flat Domains

A **flat domain** has a simple structure:
- One bottom element ⊥
- Many maximal elements (the "actual values")
- No other elements

The natural numbers form a flat domain:
```
        0   1   2   3   4   ...  (maximal elements)
         \ | / | \ | /
           ⊥              (bottom)
```

There's no "partial integer" — you either know which number it is, or you don't.

In [ ]:
# Explore flat_nat domain
flat_nat = domains_df[domains_df['domain'] == 'flat_nat']

print("The flat natural numbers domain:")
print("\nLevel 0 (bottom):")
for _, row in flat_nat[flat_nat['information_level'] == 0].iterrows():
    print(f"  {row['element']}: {row['notes']}")

print("\nLevel 1 (maximal):")
for _, row in flat_nat[flat_nat['information_level'] == 1].iterrows():
    print(f"  {row['element']}: {row['notes']}")

print("\nOrdering: ⊥ ⊑ 0, ⊥ ⊑ 1, ⊥ ⊑ 2, ... but 0 ⋢ 1 and 1 ⋢ 0")

## 4. Non-Flat Domains: Lazy Lists

Some domains have richer structure. **Lazy lists** can be partially defined:

- `⊥` — completely undefined
- `1:⊥` — we know the head is 1, but the tail is unknown
- `1:2:⊥` — we know the first two elements
- `1:2:[]` — we know the complete list [1,2]

This forms a chain: `⊥ ⊑ 1:⊥ ⊑ 1:2:⊥ ⊑ 1:2:[]`

In [ ]:
# Explore lazy_list domain
lazy_list = domains_df[domains_df['domain'] == 'lazy_list'].sort_values('information_level')

print("The lazy list domain (partial lists):")
for _, row in lazy_list.iterrows():
    maximal = "(maximal)" if row['is_maximal'] else ""
    approx = f"⊑ {row['approximates']}" if pd.notna(row['approximates']) else ""
    print(f"  Level {row['information_level']}: {row['element']} {maximal}")
    print(f"      {row['notes']}")
    if approx:
        print(f"      Approximated by: {row['approximates']}")

## 5. Approximation Chains

In lazy lists, we can have infinite approximation chains:

```
⊥ ⊑ 1:⊥ ⊑ 1:1:⊥ ⊑ 1:1:1:⊥ ⊑ ... ⊑ [1,1,1,...]
```

Each step adds more information. The limit is an **infinite list of ones**.

This is crucial for understanding recursion: a recursive definition can be seen as the limit of such a chain.

In [ ]:
# Simulate approximation chain
def approx_ones(n):
    """Return the n-th approximation to the infinite list of ones."""
    if n == 0:
        return "⊥"
    return ":".join(["1"] * n) + ":⊥"

print("Approximations to the infinite list of ones:")
for i in range(6):
    print(f"  Step {i}: {approx_ones(i)}")
print("  ...")
print("  Limit: [1,1,1,1,1,...] (infinite list)")

## 6. Interval Domains

Another example: **interval domains** for real numbers.

- `[0,1]` — the value is somewhere between 0 and 1
- `[0.4,0.6]` — more precise: between 0.4 and 0.6
- `0.5` — exact value (maximal element)

Narrower intervals have more information: `[0,1] ⊑ [0.4,0.6] ⊑ 0.5`

In [ ]:
# Explore interval domain
interval = domains_df[domains_df['domain'] == 'interval_domain'].sort_values('information_level')

print("The interval domain (for real numbers):")
for _, row in interval.iterrows():
    maximal = "(maximal)" if row['is_maximal'] else ""
    print(f"  Level {row['information_level']}: {row['element']} {maximal}")
    print(f"      {row['notes']}")

## 7. Maximal Elements

A **maximal element** is one that cannot be further refined — it contains complete information.

In flat domains, all non-bottom elements are maximal.
In lazy lists, only finite complete lists are maximal.

Infinite lists are **limits** but not maximal in the usual sense.

In [ ]:
# Count maximal vs non-maximal elements
maximal = domains_df[domains_df['is_maximal'] == True]
non_maximal = domains_df[domains_df['is_maximal'] == False]

print("Maximal elements (complete information):")
for _, row in maximal.head(8).iterrows():
    print(f"  {row['domain']}: {row['element']}")

print("\nNon-maximal elements (partial information):")
for _, row in non_maximal.iterrows():
    print(f"  {row['domain']}: {row['element']}")

## 8. Why This Matters for Recursion

The information ordering gives us a way to understand recursive definitions:

1. Start with ⊥ (no information)
2. Apply the recursive definition once → get some information
3. Apply it again → get more information
4. Take the limit → get the full answer

This is the key to Mott's fixed-point theory, which we'll see in Tutorial 5.

In [ ]:
# Preview: factorial as a limit of approximations
print("Preview: factorial as increasing approximations")
print()
print("  f₀ = ⊥                      (undefined everywhere)")
print("  f₁ = {0 → 1, else → ⊥}      (knows factorial(0))")
print("  f₂ = {0 → 1, 1 → 1, else → ⊥}")
print("  f₃ = {0 → 1, 1 → 1, 2 → 2, else → ⊥}")
print("  ...")
print("  f∞ = factorial              (the least fixed point)")

## Summary

In this tutorial, we learned:

1. **Partial orders** are relations that order elements by information content
2. **⊥ (bottom)** represents no information / non-termination
3. **Flat domains** have only ⊥ and maximal elements (no partial values)
4. **Lazy lists** have rich structure with partial lists
5. **Approximation chains** show how information increases step by step
6. **Maximal elements** contain complete information

In the next tutorial, we'll see what it means for a partial order to be a **domain** (dcpo).

---

## Exercises

1. Draw the Hasse diagram for the flat boolean domain {⊥, true, false}.

2. What are the first five approximations to the infinite list [0,1,2,3,4,...]?

3. Is `[1,⊥] ⊑ [1,2,3]`? Why or why not?

4. Can a partial order have two different bottom elements? Why not?

---

*Next: Tutorial 3 - Domains and Completeness*